In [82]:
import pandas as pd
from functools import reduce #necessary to create the last dataframe
df = pd.read_csv(r"C:\Users\hp pc\Desktop\Tesi\Smart cities\Databases\Vari\Cities Population\TotPop.csv", encoding = "ISO-8859-1",index_col=0)
df["Time"]=df.index # Create a column with the date which is set as index by default
df=df[df.Value != ':'] #select dfpopT where values in certain column are not ":"
df["Value"]=[int(s.replace(" ","")) for s in df["Value"]] # convert Value column into int 

# Rename cities in English
df = df.replace(["Athina"],"Athens")
df = df.replace(['Bruxelles / Brussel'],'Bruxelles')
df = df.replace(['København'],'Copenhagen')
df = df.replace(['Helsinki / Helsingfors'],'Helsinki')
df = df.replace(['Madrid'],'Madrid')
df = df.replace(['Paris (greater city)'],'Paris') #(In this case Eurostat database inverted data with the greater city)
df = df.replace(['Praha'],'Prague')
df = df.replace(['Roma'],'Rome')
df = df.replace(['Stockholm'],'Stockholm')
df = df.replace(['Warszawa'],'Warsaw')
df = df.replace(['Wien'],'Vienna')
df = df.replace(['Zagreb'],'Zagreb')

# CREATE A LIST OF THE CITIES
cities=["Athens", "Bruxelles", "Copenhagen", "Helsinki", "Madrid","Paris", "Prague","Rome", 
        "Stockholm", "Vienna", "Warsaw", "Zagreb",]

df.sort_values(by=['CITIES'], inplace=True, ascending=True) # sort by cities
df.reset_index(drop=True, inplace=True)  # reset index
indicators = list(dict.fromkeys(df["INDIC_UR"])) # make list containing names of the variables (no duplicates)

unav_cities=[] # create an empty list that will be filled with cities which don't have available data
dfList=[] #create an empty list that will filled with dataframes

for nome_indicatore in indicators:
    df1=df[df.INDIC_UR == nome_indicatore]
    for city in cities: 
    #create a subdataframe for each indicator
        sub_df=df1[df1.CITIES == city] #take the subdataframe with the name of each city 
        sub_df=sub_df[sub_df.Time==max(sub_df["Time"])] # get the row where date is the most recent
        dfList.append(sub_df)
df=(pd.concat(dfList,ignore_index=False)) # This df containes all variables: it needs to be splitted...

final_dfs=[]     #...split the previous df and creating the complete one
for indicator in indicators:
    df1=df[df.INDIC_UR == indicator] 
    df1 = df1.rename(columns={"Value": indicator}) #rename "Value" with the name of the variable 
    del df1["INDIC_UR"] # delete the column filled with the name of the variable. 
    df1.reset_index(drop=True, inplace=True) 
    df1.sort_values("CITIES", axis=0, ascending=True, inplace=True)
    final_dfs.append(df1) # this is the list of df that need to be merged
df= reduce(lambda x, y: pd.merge(x, y, on = 'CITIES'), final_dfs) #merge dfs on CITIES one next to the other
del df["Time_y"] # (before deleting check if columns are equals)
del df["Time_x"]
df.columns=["CITIES","pop_tot","pop_female",]
df["%_women_pop"]= round((df["pop_female"]/df["pop_tot"])*100,2)
df

,CITIES,pop_tot,pop_female,%_women_pop
0,Athens,664046,348836,52.53
1,Bruxelles,1215289,618282,50.88
2,Copenhagen,559440,283184,50.62
3,Helsinki,648042,340409,52.53
4,Madrid,3266126,1745973,53.46
5,Paris,6754282,3518746,52.10
6,Prague,1324277,676991,51.12
7,Rome,2856133,1504189,52.67
8,Stockholm,949761,480264,50.57
9,Vienna,1766746,916150,51.86


In [83]:
dfpopulation_cities=df
%store dfpopulation_cities

Stored 'dfpopulation_cities' (DataFrame)
